In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd
import os

# Path to the folder containing CSV files
folder_path = '/content/drive/My Drive/Preprocessed'

# Combine all CSV files into one DataFrame
all_data = pd.DataFrame()

for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):  # Ensure it's a CSV file
        file_path = os.path.join(folder_path, file_name)
        # Specify the semicolon as the delimiter
        temp_data = pd.read_csv(file_path, sep=';')
        all_data = pd.concat([all_data, temp_data], ignore_index=True)

# Display the combined dataset
print("Combined DataFrame:")
print(all_data.info())
print(all_data.head())


In [ ]:
import pandas as pd

# Load one of the CSV files for inspection
file_path = '/content/drive/My Drive/Preprocessed/HUPA0015P.csv'  # Replace with the actual path
data = pd.read_csv(file_path, sep=';', header=None)  # Use sep=';' to handle semicolon delimiter

# Display the first few rows
print(data.head())

In [ ]:
data = pd.read_csv(file_path, sep=';', header=0, engine='python')  # Use 'python' engine for better compatibility
print(data.columns)
print(data.head())


In [ ]:
import os

# Path to the folder
folder_path = '/content/drive/My Drive/Preprocessed'

all_data = pd.DataFrame()

for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)

        # Read and split the data
        temp_data = pd.read_csv(file_path, header=None)
        temp_data_split = temp_data[0].str.split(';', expand=True)
        temp_data_split.columns = ['time', 'glucose', 'calories', 'heart_rate', 'steps', 'basal_rate', 'bolus_volume_delivered', 'carb_input']

        # Combine into the main DataFrame
        all_data = pd.concat([all_data, temp_data_split], ignore_index=True)

print(all_data.info())
print(all_data.head())


In [ ]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, confusion_matrix, precision_score, recall_score, accuracy_score, f1_score


# Combine all preprocessed CSV files
preprocessed_dir = '/content/drive/My Drive/Preprocessed'
all_data = pd.DataFrame()

for file_name in os.listdir(preprocessed_dir):
    file_path = os.path.join(preprocessed_dir, file_name)
    # Read and split the data, assuming semicolon delimiter and no header
    temp_data = pd.read_csv(file_path, header=None)
    temp_data_split = temp_data[0].str.split(';', expand=True)
    temp_data_split.columns = ['time', 'glucose', 'calories', 'heart_rate', 'steps', 'basal_rate', 'bolus_volume_delivered', 'carb_input']
    all_data = pd.concat([all_data, temp_data_split], ignore_index=True)

# Separate features and target
target_column = 'glucose'
X = all_data.drop(columns=[target_column, 'time'])
y = all_data[target_column]

# Convert columns to numeric, handling errors
X = X.apply(pd.to_numeric, errors='coerce').fillna(0)  # Replace non-numeric with 0
y = pd.to_numeric(y, errors='coerce').fillna(0)  # Replace non-numeric with 0

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


# --- BRC Model ---
# Create random reservoirs for forward and backward directions
reservoir_size = 500
reservoir_forward = np.random.rand(X_train.shape[1], reservoir_size) * 2 - 1
reservoir_backward = np.random.rand(X_train.shape[1], reservoir_size) * 2 - 1

# Transform training and testing data using the forward reservoir
reservoir_train_forward = np.tanh(np.dot(X_train, reservoir_forward))
reservoir_test_forward = np.tanh(np.dot(X_test, reservoir_forward))

# Reverse the input data for the backward reservoir
X_train_reversed = np.flip(X_train, axis=0)
X_test_reversed = np.flip(X_test, axis=0)

# Transform reversed data using the backward reservoir
reservoir_train_backward = np.tanh(np.dot(X_train_reversed, reservoir_backward))
reservoir_test_backward = np.tanh(np.dot(X_test_reversed, reservoir_backward))

# Flip the backward states back to the original order
reservoir_train_backward = np.flip(reservoir_train_backward, axis=0)
reservoir_test_backward = np.flip(reservoir_test_backward, axis=0)

# Combine forward and backward reservoir states
reservoir_train_combined = np.hstack((reservoir_train_forward, reservoir_train_backward))
reservoir_test_combined = np.hstack((reservoir_test_forward, reservoir_test_backward))

# Handle NaN values in combined reservoir states and target (if any)
reservoir_train_combined = np.nan_to_num(reservoir_train_combined)
reservoir_test_combined = np.nan_to_num(reservoir_test_combined)

# Train the readout layer using Ridge Regression
ridge = Ridge(alpha=1.0)
ridge.fit(reservoir_train_combined, y_train)

# Evaluate and calculate metrics
predictions_brc = ridge.predict(reservoir_test_combined)

# --- Regression Metrics ---
mse_brc = mean_squared_error(y_test, predictions_brc)
mae_brc = mean_absolute_error(y_test, predictions_brc)
r2_brc = r2_score(y_test, predictions_brc)

print("BRC Regression Metrics:")
print(f"Mean Squared Error (MSE): {mse_brc}")
print(f"Mean Absolute Error (MAE): {mae_brc}")
print(f"R-squared (R2): {r2_brc}")

# --- Categorization for Classification Metrics ---
thresholds = [70, 140]  # Example thresholds
y_test_cat = np.digitize(y_test, thresholds)
predictions_brc_cat = np.digitize(predictions_brc, thresholds)

# --- Classification Metrics ---
cm_brc = confusion_matrix(y_test_cat, predictions_brc_cat)
precision_brc = precision_score(y_test_cat, predictions_brc_cat, average='weighted')
recall_brc = recall_score(y_test_cat, predictions_brc_cat, average='weighted')
accuracy_brc = accuracy_score(y_test_cat, predictions_brc_cat)
f1_brc = f1_score(y_test_cat, predictions_brc_cat, average='weighted')

print("\nBRC Classification Metrics:")
print("Confusion Matrix:", cm_brc)
print("Precision:", precision_brc)
print("Recall:", recall_brc)
print("Accuracy:", accuracy_brc)
print("F1-score:", f1_brc)